In [ ]:
import nibabel as nib
from totalsegmentator.python_api import totalsegmentator
import os
import SimpleITK as sitk
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
from image_utils import convert_series_to_nifti, quantize_maps, quantize_maps_top_quarter, clear_directory, convert_and_copy_with_labels_and_rename, generate_dataset_json
import logging
from image_utils import ImageVisualizer, show_progress
from skimage.restoration import inpaint

DATASET_NUM = "047"
FOLD_NUM = 0

In [ ]:
input_folder = r'D:\CTH_archive\CTA_NIFTI'
stripped_folder = r'D:\CTH_archive\CTA_STRIPPED_MASK'
output_folder = r'D:\CTH_archive\CTA_STRIPPED'


os.makedirs(stripped_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Skull stripping process
for filename in os.listdir(input_folder):
    if filename.endswith(('.nii')):
        patient_name = filename.split(' ')[0]  # Extract patient name
        input_path = os.path.join(input_folder, filename)
        stripped_path = os.path.join(stripped_folder, patient_name)

        # Check if the stripped file already exists
        if not os.path.exists(stripped_path):
            try:
                input_img = nib.load(input_path)
                totalsegmentator(input_img, stripped_path, roi_subset=['brain'], ml=True)
                print(f"Processed and saved: {stripped_path}")
            except Exception as e:
                print(f"Error processing {input_path}: {e}")
        else:
            print(f"Stripped file already exists, skipping: {stripped_path}")

# Applying masks to stripped images
missing_files = []

for mask_filename in tqdm(os.listdir(stripped_folder)):
    base_filename = os.path.splitext(mask_filename)[0].rsplit('.', 1)[0]  # Handle potential double extensions
    original_path = None

    # Find matching original file
    for original_filename in os.listdir(input_folder):
        if os.path.splitext(original_filename)[0].rsplit('.', 1)[0] == base_filename:
            original_path = os.path.join(input_folder, original_filename)
            break

    if original_path:
        mask_path = os.path.join(stripped_folder, mask_filename)

        output_filename = base_filename
        output_path = os.path.join(output_folder, output_filename)

        if not os.path.exists(output_path):
            print(f"Applying mask to {original_path} using {mask_path}")

            # Load and apply mask
            mask_nii = nib.load(mask_path)
            original_nii = nib.load(original_path)
            masked_data = np.where(mask_nii.get_fdata() > 0, original_nii.get_fdata(), 0)
            masked_nii = nib.Nifti1Image(masked_data, affine=original_nii.affine)

            nib.save(masked_nii, output_path)

        else:
            print(f"Masked file already exists, skipping: {output_path}")
    else:
        print(f"No matching file found for {mask_filename} in {input_folder}")
        missing_files.append((mask_filename, input_folder))

print("Mask application complete.")
if missing_files:
    print("Missing files:", missing_files)


In [ ]:
transforms_dir =  "/mnt/d/CTH_archive/CTH_TMAX_TRANSFORMS_COLOR"
fixed_images_dir = "/mnt/d/CTH_archive/CTH_STRIPPED"  
moving_images_dir = "/mnt/d/CTH_archive/CTP_STRIPPED" 
transformed_img_dir = "/mnt/d/CTH_archive/CTH_STRIPPED_REG_COLOR"

def register_images(fixed_image_path, moving_image_path, transforms_dir):
    # Extract patient identifier from the file name, ensuring .nii is not included
    patient = os.path.splitext(os.path.basename(moving_image_path))[0]
    patient = os.path.splitext(patient)[0]  # Remove .nii if present

    # Construct the transform file path
    transform_file = os.path.join(transforms_dir, f'{patient}.h5')

    # Check if the transform file already exists and skip registration if it does
    if os.path.exists(transform_file):
        print(f"Transform file already exists for patient {patient}, skipping registration.")
        return

    # Load the fixed and moving images
    fixed_image = sitk.ReadImage(fixed_image_path)
    moving_image = sitk.ReadImage(moving_image_path)

    # Initialize the registration method
    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingPercentage(0.10)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetOptimizerAsGradientDescentLineSearch(learningRate=0.5, numberOfIterations=500)
    registration_method.SetOptimizerScalesFromPhysicalShift()
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors=[8, 4, 2])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[4, 2, 1])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    initial_transform = sitk.CenteredTransformInitializer(sitk.Cast(fixed_image, moving_image.GetPixelID()), 
                                                      moving_image, 
                                                      sitk.AffineTransform(fixed_image.GetDimension()),
                                                      sitk.CenteredTransformInitializerFilter.MOMENTS)
    
    registration_method.SetInitialTransform(initial_transform, True)

    try:
        # Execute the registration
        final_transform = registration_method.Execute(fixed_image, moving_image)
        
        # Save the transform
        sitk.WriteTransform(final_transform, transform_file)
        sitk.WriteImage(sitk.Resample(moving_image, fixed_image, final_transform,  sitk.sitkNearestNeighbor), f"{transformed_img_dir}/{patient}.nii")
        print("Final metric value: {0}".format(registration_method.GetMetricValue()))
        print(
            "Optimizer's stopping condition, {0}".format(
                registration_method.GetOptimizerStopConditionDescription()
            )
        )
        print(f"Registration successful for patient: {patient}. Transform saved to {transform_file}")
    except RuntimeError as e:
        print(f"Registration failed for patient {patient}: {e}")


for filename in tqdm(os.listdir(fixed_images_dir)):
    fixed_image_path = os.path.join(fixed_images_dir, filename)
    moving_image_path = os.path.join(moving_images_dir, filename)

    if os.path.isfile(fixed_image_path) and os.path.isfile(moving_image_path):
        if not os.path.exists(transforms_dir):
            os.makedirs(transforms_dir, exist_ok=True)
        register_images(fixed_image_path, moving_image_path, transforms_dir)


In [ ]:
import SimpleITK as sitk
import os
import logging
import numpy as np

FINAL_SIZE = 512

def resample_image(moving_image, ctp_image):
    desired_size = [FINAL_SIZE, FINAL_SIZE, ctp_image.GetSize()[2]]  # Use the same number of slices as the CTP image
    num_channels = moving_image.GetNumberOfComponentsPerPixel()
    
    resampled_channels = []
    for channel in range(num_channels):
        channel_image = sitk.VectorIndexSelectionCast(moving_image, channel)
        
        resampler = sitk.ResampleImageFilter()
        resampler.SetReferenceImage(channel_image)
        resampler.SetSize(desired_size)
        resampler.SetOutputSpacing([channel_image.GetSpacing()[i] * (channel_image.GetSize()[i] / desired_size[i]) for i in range(3)])
        resampler.SetInterpolator(sitk.sitkLinear)
        
        resized_channel = resampler.Execute(channel_image)
        resized_channel.SetSpacing(ctp_image.GetSpacing())
        resized_channel.SetOrigin(ctp_image.GetOrigin())
        resized_channel.SetDirection(ctp_image.GetDirection())
        
        resampled_channels.append(resized_channel)
    
    resized_moving_image = sitk.Compose(resampled_channels)
    return resized_moving_image

def correct_orientation(image, reference_image):
    """
    Correct the orientation of the image to match the reference image.
    """
    # Get the direction matrices
    image_direction = np.array(image.GetDirection()).reshape(3, 3)
    reference_direction = np.array(reference_image.GetDirection()).reshape(3, 3)
    
    # Check if the directions are different
    if not np.allclose(image_direction, reference_direction):
        # Calculate the transformation to align the orientations
        transform = sitk.AffineTransform(3)
        transform.SetMatrix(np.dot(reference_direction, np.linalg.inv(image_direction)).flatten())
        
        # Apply the transformation
        corrected_image = sitk.Resample(image, reference_image, transform, sitk.sitkLinear, 0.0, image.GetPixelID())
        
        # Ensure the corrected image has the same metadata as the reference
        corrected_image.SetOrigin(reference_image.GetOrigin())
        corrected_image.SetSpacing(reference_image.GetSpacing())
        corrected_image.SetDirection(reference_image.GetDirection())
        
        return corrected_image
    else:
        return image

def apply_final_transform(resized_moving_image, fixed_image, transform_file_path):
    try:
        final_transform = sitk.ReadTransform(transform_file_path)
        
        if resized_moving_image:
            num_channels = resized_moving_image.GetNumberOfComponentsPerPixel()
            resampled_channels = []
            
            for channel in range(num_channels):
                channel_image = sitk.VectorIndexSelectionCast(resized_moving_image, channel)
                
                resampled_channel = sitk.Resample(channel_image, 
                                                  fixed_image, 
                                                  final_transform, 
                                                  sitk.sitkLinear, 
                                                  0.0, 
                                                  channel_image.GetPixelID())
                
                resampled_channels.append(resampled_channel)
            
            resampled_image = sitk.Compose(resampled_channels)
            
            # Correct orientation if needed
            resampled_image = correct_orientation(resampled_image, fixed_image)
            
            return resampled_image
        else:
            logging.error("Resized moving image is invalid. Cannot apply final transform.")
            return None
    except Exception as e:
        logging.error("Failed to apply final transform: " + str(e))
        return None

# Main processing
def main():
    tmax_nifti_dir = r"/mnt/d/CTH_archive/TMAX_NIFTI_512"
    transforms_dir = r"/mnt/d/CTH_archive/CTH_TMAX_TRANSFORMS_COLOR"
    cth_stripped_dir = r"/mnt/d/CTH_archive/CTH_STRIPPED"
    ctp_stripped_dir = r"/mnt/d/CTH_archive/CTP_STRIPPED"
    
    output_dir = r"/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/TMAX_NIFTI_REG"
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over the transformation files
    for transform_file in os.listdir(transforms_dir):
        transform_path = os.path.join(transforms_dir, transform_file)
        base_filename = transform_file.replace('.h5', '')

        # Paths to the moving, fixed, and reference (ctp) images
        moving_image_path = os.path.join(tmax_nifti_dir, base_filename + '.nii')
        fixed_image_path = os.path.join(cth_stripped_dir, base_filename + '.nii')
        ctp_image_path = os.path.join(ctp_stripped_dir, base_filename + '.nii')
        print(f"Processing {base_filename}...")

        # Check if all required files exist
        if os.path.exists(moving_image_path) and os.path.exists(fixed_image_path) and os.path.exists(ctp_image_path):
            # Load images
            moving_image = sitk.ReadImage(moving_image_path)
            fixed_image = sitk.ReadImage(fixed_image_path)
            ctp_image = sitk.ReadImage(ctp_image_path)

            resized_moving_image = resample_image(moving_image, ctp_image)

            # Apply the final transformation
            resampled_image = apply_final_transform(resized_moving_image, fixed_image, transform_path)

            # Save the resampled image
            resampled_image_path = os.path.join(output_dir, base_filename + '.nii')
            sitk.WriteImage(resampled_image, resampled_image_path)
            print(f"Processed and saved: {resampled_image_path}")
        else:
            print(f"Required files for {base_filename} are not available.")

if __name__ == "__main__":
    main()

In [ ]:
import os
import logging
import SimpleITK as sitk
import numpy as np
from matplotlib import cm

FINAL_SIZE = 512  # Assuming you want to keep this constant

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

import os
import numpy as np
from matplotlib import cm
import SimpleITK as sitk

import SimpleITK as sitk
import numpy as np

def process_3d_rgb_image(image):
    try:
        # Convert SimpleITK image to numpy array
        array = sitk.GetArrayFromImage(image)
        
        # Ensure the array is in the correct shape (depth, height, width, channels)
        if array.shape[-1] != 3:
            array = np.moveaxis(array, 0, -1)
        
        # Normalize the array to 0-255 range if it's not already
        if array.max() <= 1.0:
            array = (array * 255).astype(np.uint8)
        
        # Convert back to SimpleITK image (RGB)
        processed_image = sitk.GetImageFromArray(array, isVector=True)
        
        # Set metadata for the new image
        processed_image.SetSpacing(image.GetSpacing())
        processed_image.SetDirection(image.GetDirection())
        processed_image.SetOrigin(image.GetOrigin())
        
        return processed_image
    except Exception as e:
        logging.error(f"Error in process_3d_rgb_image: {str(e)}")
        logging.error(f"Image shape: {image.GetSize()}, Pixel type: {image.GetPixelID()}")
        logging.exception("Stack trace:")
        raise
    
def resample_image(moving_image, ctp_image):
    desired_size = [FINAL_SIZE, FINAL_SIZE, ctp_image.GetSize()[2]]  # Use the same number of slices as the CTP image
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(moving_image)
    resampler.SetSize(desired_size)
    resampler.SetOutputSpacing([moving_image.GetSpacing()[i] * (moving_image.GetSize()[i] / desired_size[i]) for i in range(3)])
    resampler.SetInterpolator(sitk.sitkLinear)
    
    resized_image = resampler.Execute(moving_image)
    resized_image.SetSpacing(ctp_image.GetSpacing())
    resized_image.SetOrigin(ctp_image.GetOrigin())
    resized_image.SetDirection(ctp_image.GetDirection())
    
    return resized_image

def correct_orientation(image, reference_image):
    image_direction = np.array(image.GetDirection()).reshape(3, 3)
    reference_direction = np.array(reference_image.GetDirection()).reshape(3, 3)
    
    if not np.allclose(image_direction, reference_direction):
        transform = sitk.AffineTransform(3)
        transform.SetMatrix(np.dot(reference_direction, np.linalg.inv(image_direction)).flatten())
        
        corrected_image = sitk.Resample(image, reference_image, transform, sitk.sitkLinear, 0.0, image.GetPixelID())
        
        corrected_image.SetOrigin(reference_image.GetOrigin())
        corrected_image.SetSpacing(reference_image.GetSpacing())
        corrected_image.SetDirection(reference_image.GetDirection())
        
        return corrected_image
    else:
        return image

def apply_final_transform(resized_moving_image, fixed_image, transform_file_path):
    try:
        final_transform = sitk.ReadTransform(transform_file_path)
        
        if resized_moving_image is not None:
            resampled_image = sitk.Resample(resized_moving_image, 
                                            fixed_image, 
                                            final_transform, 
                                            sitk.sitkBSpline, 
                                            0.0, 
                                            resized_moving_image.GetPixelID())
            
            resampled_image = correct_orientation(resampled_image, fixed_image)
            
            return resampled_image
        else:
            logging.error("Resized moving image is invalid. Cannot apply final transform.")
            return None
    except Exception as e:
        logging.error(f"Failed to apply final transform: {str(e)}")
        return None

def main():
    tmax_nifti_dir = r"/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/TMAX_NIFTI_Processed"
    transforms_dir = r"/mnt/d/CTH_archive/CTH_TMAX_TRANSFORMS_COLOR"
    cth_stripped_dir = r"/mnt/d/CTH_archive/CTH_STRIPPED"
    ctp_stripped_dir = r"/mnt/d/CTH_archive/CTP_STRIPPED"
    
    output_dir = r"/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/TMAX_NIFTI_REG"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for transform_file in os.listdir(transforms_dir):
        transform_path = os.path.join(transforms_dir, transform_file)
        base_filename = transform_file.replace('.h5', '')

        moving_image_path = os.path.join(tmax_nifti_dir, base_filename + '.nii')
        fixed_image_path = os.path.join(cth_stripped_dir, base_filename + '.nii')
        ctp_image_path = os.path.join(ctp_stripped_dir, base_filename + '.nii')
        logging.info(f"Processing {base_filename}...")

        if os.path.exists(moving_image_path) and os.path.exists(fixed_image_path) and os.path.exists(ctp_image_path):
            try:
                moving_image = sitk.ReadImage(moving_image_path)
                fixed_image = sitk.ReadImage(fixed_image_path)
                ctp_image = sitk.ReadImage(ctp_image_path)

                logging.info(f"Moving image: size={moving_image.GetSize()}, pixel type={moving_image.GetPixelID()}")
                logging.info(f"Fixed image: size={fixed_image.GetSize()}, pixel type={fixed_image.GetPixelID()}")
                logging.info(f"CTP image: size={ctp_image.GetSize()}, pixel type={ctp_image.GetPixelID()}")

                resized_moving_image = resample_image(moving_image, ctp_image)
                logging.info(f"Resized moving image: size={resized_moving_image.GetSize()}, pixel type={resized_moving_image.GetPixelID()}")

                resampled_image = apply_final_transform(resized_moving_image, fixed_image, transform_path)

                if resampled_image is not None:
                    logging.info(f"Resampled image: size={resampled_image.GetSize()}, pixel type={resampled_image.GetPixelID()}")
                    logging.info(f"Resampled image origin: {resampled_image.GetOrigin()}")
                    logging.info(f"Resampled image spacing: {resampled_image.GetSpacing()}")
                    logging.info(f"Resampled image direction: {resampled_image.GetDirection()}")
                    
                    # Process the 3D RGB image
                    processed_image = process_3d_rgb_image(resampled_image)
                    
                    output_image_path = os.path.join(output_dir, base_filename + '.nii')
                    sitk.WriteImage(processed_image, output_image_path)
                    logging.info(f"Processed and saved: {output_image_path}")
                else:
                    logging.error(f"Failed to process {base_filename}. Skipping.")
            except Exception as e:
                logging.error(f"Error processing {base_filename}: {str(e)}")
                logging.exception("Stack trace:")
        else:
            logging.warning(f"Required files for {base_filename} are not available.")

if __name__ == "__main__":
    main()

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np

# Load the NIFTI files
name = "BATTLE_MARIA"
file1_path = f"/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/TMAX_CTA_NIFTI_REG/{name}.nii"
file2_path = f"/mnt/d/CTH_archive/CTA_STRIPPED/{name}.nii"

nii1 = nib.load(file1_path)
nii2 = nib.load(file2_path)

img1 = nii1.get_fdata()
img2 = nii2.get_fdata()

# Ensure images are in the correct shape
img1 = np.squeeze(img1)
img2 = np.squeeze(img2)

# Function to display slices
def display_slices(img1, img2, slices):
    fig, axes = plt.subplots(len(slices), 3, figsize=(15, 5 * len(slices)))
    for i, slice_idx in enumerate(slices):
        axes[i, 0].imshow(img1[:, :, slice_idx], cmap='gray')
        axes[i, 0].set_title(f'Image 1 - Slice {slice_idx}')
        axes[i, 1].imshow(img2[:, :, slice_idx], cmap='gray')
        axes[i, 1].set_title(f'Image 2 - Slice {slice_idx}')
        axes[i, 2].imshow(img1[:, :, slice_idx], cmap='gray')
        axes[i, 2].imshow(img2[:, :, slice_idx], cmap='hot', alpha=0.5)
        axes[i, 2].set_title(f'Overlay - Slice {slice_idx}')
    plt.tight_layout()
    plt.show()

# Display a few slices for comparison
slices_to_display = [5, 10, 50, 70]  # You can adjust these indices based on your data
display_slices(img1, img2, slices_to_display)


In [ ]:
def convert_series_to_nifti(input_directory, output_file):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(input_directory)
    reader.SetFileNames(dicom_names)
    image_series = reader.Execute()
    
    # Convert to numpy array to manipulate the pixel data directly
    img_array = sitk.GetArrayFromImage(image_series)
    
    #Need to get rid of the title text and idk how to access the pxiel data so here is a hacky way to do it
    
    # Identify non-black pixel locations in the first 33 rows of the first slice
    non_black_pixels = img_array[0, :33, :] > 0
    
    # Black out these pixels in the first 33 rows across all slices
    for i in range(img_array.shape[0]):
        img_array[i, :33, :][non_black_pixels] = 0

        # Apply inpainting to the blacked-out area
        mask = (img_array[i, :33, :] == 0) & non_black_pixels  # Create a mask where inpainting is needed
        img_array[i, :33, :] = inpaint.inpaint_biharmonic(img_array[i, :33, :], mask)
    
    # Check if the image is color (3 channels)
    if img_array.ndim == 4 and img_array.shape[-1] == 3:
        # Convert RGB to Grayscale using the luminosity method
        img_array = np.dot(img_array[..., :3], [0.2989, 0.5870, 0.1140])
        
    # Convert the numpy array back to a SimpleITK Image
    processed_image = sitk.GetImageFromArray(img_array)
    processed_image.SetSpacing(image_series.GetSpacing())
    processed_image.SetOrigin(image_series.GetOrigin())
    processed_image.SetDirection(image_series.GetDirection())

    # Write the processed image as a NIfTI file
    sitk.WriteImage(processed_image, output_file)


# Directory paths
input_dir = 'D:\\CTH_archive\\CBF'
output_dir = 'D:\\CTH_archive\\CBF_NIFTI'

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop through each directory in the input directory
for patient_name in tqdm(os.listdir(input_dir)):
    patient_path = os.path.join(input_dir, patient_name)
    if os.path.isdir(patient_path):
        # Assume the patient name should be used for the output file
        output_file = os.path.join(output_dir, f"{patient_name}.nii")
        convert_series_to_nifti(patient_path, output_file)


In [ ]:
TMAX_SOURCE_DIR = r'D:\CTH_archive\CBF_REGISTERED'
TMAX_QUNTIZED_DIR = r'D:\CTH_archive\CBF_NIFTI_QUANT_BIN_REGISTERED'
from image_utils import convert_series_to_nifti, quantize_maps, quantize_maps_top_quarter, clear_directory, convert_and_copy_with_labels_and_rename, generate_dataset_json
#quantization_levels = 5
#quantize_maps(TMAX_SOURCE_DIR, TMAX_QUNTIZED_DIR, quantization_levels=quantization_levels)

quantize_maps_top_quarter(TMAX_SOURCE_DIR, TMAX_QUNTIZED_DIR)

In [ ]:
#Create Dataset for nnUNet

image_source_dir = rf"D:/CTH_archive/CTA_STRIPPED/"
label_source_dir = rf"D:/CTH_archive/CBF_NIFTI_QUANT_BIN_REGISTERED"

image_target_dir = rf"D:/nnUNet_raw/Dataset{DATASET_NUM}_Perfusion/imagesTr"
label_target_dir = rf"D:/nnUNet_raw/Dataset{DATASET_NUM}_Perfusion/labelsTr"

# Define target directories for testing
images_test_dir = rf"D:/nnUNet_raw/Dataset{DATASET_NUM}_Perfusion/imagesTs"
labels_test_dir = rf"D:/nnUNet_raw/Dataset{DATASET_NUM}_Perfusion/labelsTs"

convert_and_copy_with_labels_and_rename(image_source_dir, image_target_dir, label_source_dir, label_target_dir, images_test_dir, labels_test_dir)

dataset_dir = rf'D:\nnUNet_raw\Dataset{DATASET_NUM}_Perfusion\labelsTr'  # Path to dataset directory
nnUNet_dir = rf'D:\nnUNet_raw\Dataset{DATASET_NUM}_Perfusion'  # Path to nnUNet directory
num_quant_levels = 5  # Number of quantization levels (excluding background)
channel_names = {"0": "CT"}
num_test_data = 5  # Specify the number of test data

generate_dataset_json(dataset_dir, num_quant_levels, channel_names, nnUNet_dir, file_ending=".nii.gz", num_test_data=num_test_data)


In [ ]:
# This is how you would run the first fold of nnUnet on the dataset, prob better to run this in a terminal instead of a notebook
!nnuNetv2_plan_and_preprocess -d $DATASET_NUM --verify_dataset_integrity
!nnUNetv2_train $DATASET_NUM 3d_fullres $FOLD_NUM --npz -device cuda
!nnUNetv2_predict -d Dataset_{DATASET_NUM}_Perfusion -i  D:\nnUNet_raw\Dataset_{DATASET_NUM}Perfusion\imagesTs -o D:\nnUNet_output\Dataset{DATAST_NUM}Perfusion -f  0 -tr nnUNetTrainer -c 3d_fullres -p nnUNetPlans

In [ ]:
#Here's a way to visualise 3 studies at at time
from image_utils import ImageVisualizer

prediction_dir = rf"D:\nnUnet_output\Dataset{DATASET_NUM}_Perfusion"
ground_truth_dir = rf"D:\nnUNet_raw\Dataset{DATASET_NUM}_Perfusion\labelsTs"
ct_images_dir = r"D:\CTH_archive\CTH_STRIPPED"

visualizer = ImageVisualizer(prediction_dir, ground_truth_dir, ct_images_dir)
visualizer.display()


In [ ]:
from image_utils import ImageVisualizer

ground_truth_dir = rf"D:\CTH_archive\CBF_NIFTI_QUANT_REGISTERED"
prediction_dir =  rf"D:\CTH_archive\CBF_NIFTI_QUANT_REGISTERED"
ct_images_dir = rf"D:\CTH_archive\CTA_STRIPPED"

visualizer = ImageVisualizer(prediction_dir, ground_truth_dir, ct_images_dir)
visualizer.display()


In [ ]:
show_progress(rf"D:\nnUNet_results\Dataset046_Perfusion\nnUNetTrainer__nnUNetPlans__3d_fullres\fold_0\progress.png")